# Download Dataset

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"haydenji","key":"78a28bd346be35cfe15d7ea2c22d9975"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                           title                                            size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------  ----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
iamsouravbanerjee/customer-shopping-trends-dataset            Customer Shopping Trends Dataset                146KB  2023-10-05 06:45:37          26485        524  1.0              
joebeachcapital/30000-spotify-songs                           30000 Spotify Songs                               3MB  2023-11-01 06:06:43           3837         88  1.0              
maso0dahmed/football-players-data                             Football Players Data                             1MB  2023-11-13 18:52:36            667         30  1.0              
alexracape/lego-sets-and-price

In [ ]:
!kaggle competitions download -c open-problems-single-cell-perturbations

100% 3.81G/3.83G [00:44<00:00, 82.7MB/s]
100% 3.83G/3.83G [00:44<00:00, 92.8MB/s]


In [ ]:
!mkdir train
!unzip open-problems-single-cell-perturbations.zip -d train

Archive:  open-problems-single-cell-perturbations.zip
  inflating: train/adata_excluded_ids.csv  
  inflating: train/adata_obs_meta.csv  
  inflating: train/adata_train.parquet  
  inflating: train/de_train.parquet  
  inflating: train/id_map.csv        
  inflating: train/multiome_obs_meta.csv  
  inflating: train/multiome_train.parquet  
  inflating: train/multiome_var_meta.csv  
  inflating: train/sample_submission.csv  


In [ ]:
import pandas as pd

In [ ]:
de_train = pd.read_parquet(f"./train/de_train.parquet")
de_train.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629


# Prepare Data

In [ ]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 57.6 MB/s eta 0:00:00


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from rdkit import Chem
from rdkit.Chem import RDKFingerprint
import numpy as np

In [ ]:
ct_df = de_train.iloc[:,0:1]

In [ ]:
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(ct_df)
ct_ohe = ohe.transform(ct_df)
ct_ohe = ct_ohe.tolist()
de_train['cell_type_onehot'] = ct_ohe
de_train.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,cell_type_onehot
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]"
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]"
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]"
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]"


In [ ]:
sm_df = de_train["SMILES"]
print(len(sm_df))
sm_df = sm_df.dropna()
print(len(sm_df))

614
614


In [ ]:
cs_arr = []
for sm in sm_df:
  try:
    cs = Chem.CanonSmiles(sm)
    cs_arr.append(cs)
  except:
    print('Invalid SMILES: ', sm)
print(len(cs_arr))

614


In [ ]:
mols = [Chem.MolFromSmiles(x) for x in cs_arr]
fps = [np.array(RDKFingerprint(mol)) for mol in mols]
de_train['fingerprint'] = fps
de_train.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,cell_type_onehot,fingerprint
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, ..."


In [ ]:
tmp = de_train.iloc[:,5:]
tmp.head()

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,cell_type_onehot,fingerprint
0,0.104720,-0.077524,-1.625596,-0.144545,0.143555,0.073229,-0.016823,0.101717,-0.005153,1.043629,...,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
1,0.915953,-0.884380,0.371834,-0.081677,-0.498266,0.203559,0.604656,0.498592,-0.317184,0.375550,...,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
2,-0.387721,-0.305378,0.567777,0.303895,-0.022653,-0.480681,0.467144,-0.293205,-0.005098,0.214918,...,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
3,0.232893,0.129029,0.336897,0.486946,0.767661,0.718590,-0.162145,0.157206,-3.654218,-0.212402,...,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
4,4.290652,-0.063864,-0.017443,-0.541154,0.570982,2.022829,0.600011,1.231275,0.236739,0.338703,...,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, ..."


In [ ]:
ge = tmp.iloc[:,:-2]
ge.head()

,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,AAK1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0.104720,-0.077524,-1.625596,-0.144545,0.143555,0.073229,-0.016823,0.101717,-0.005153,1.043629,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,0.915953,-0.884380,0.371834,-0.081677,-0.498266,0.203559,0.604656,0.498592,-0.317184,0.375550,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,-0.387721,-0.305378,0.567777,0.303895,-0.022653,-0.480681,0.467144,-0.293205,-0.005098,0.214918,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,0.232893,0.129029,0.336897,0.486946,0.767661,0.718590,-0.162145,0.157206,-3.654218,-0.212402,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,4.290652,-0.063864,-0.017443,-0.541154,0.570982,2.022829,0.600011,1.231275,0.236739,0.338703,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629


In [ ]:
ct_sm = tmp.iloc[:,-2:]
ct_sm.head()

,cell_type_onehot,fingerprint
0,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
1,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
2,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
4,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, ..."


In [ ]:
# imports
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn as nn
import time
import torch.nn.functional as F

In [ ]:
t1 = torch.tensor(ct_sm.iloc[0,0])
t2 = torch.tensor(ct_sm.iloc[0,1])
t = torch.cat((t1, t2), dim=0)
len(t)

2054

In [ ]:
de_train.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,cell_type_onehot,fingerprint
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ..."
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, ..."


# Train LightGBM

In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [ ]:
res_arr = []
for i in range(len(de_train)):
  ct_ohe = np.array(de_train.iloc[i,-2:-1][0])
  fingerprint = np.array(de_train.iloc[i,-1:][0])
  res = np.concatenate((ct_ohe, fingerprint), axis=0)
  res_arr.append(res)

In [ ]:
de_train['merged'] = res_arr

In [ ]:
de_train.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,cell_type_onehot,fingerprint,merged
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]","[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ..."


In [ ]:
X = np.array(de_train.iloc[:,-1:], dtype=object)
Y = np.array(de_train.iloc[:,5:-3], dtype=object)

In [ ]:
tmp_x = de_train.iloc[:,-1:]
arr = np.empty(len(tmp_x), dtype=object)
for i in range(len(tmp_x)):
  arr[i] = tmp_x.iloc[i,0]
X = arr
print(len(X[1]))

2054


In [ ]:
X = np.vstack(X)
X.shape

(614, 2054)

In [ ]:
tmp_y = de_train.iloc[:,5:-3]
arr = np.empty(len(tmp_y), dtype=object)
for i in range(len(tmp_y)):
  arr[i] = tmp_y.iloc[i].values
Y = arr
print(len(Y[1]))

18211


In [ ]:
tmp_y = de_train.iloc[:,5:5676]
arr = np.empty(len(tmp_y), dtype=object)
for i in range(len(tmp_y)):
  arr[i] = tmp_y.iloc[i].values
Y_1 = arr
print(len(arr[1]))

5671


In [ ]:
tmp_y = de_train.iloc[:,10005:-3]
arr = np.empty(len(tmp_y), dtype=object)
for i in range(len(tmp_y)):
  arr[i] = tmp_y.iloc[i].values
Y_2 = arr
print(len(arr[1]))

8211


In [ ]:
Y = np.vstack(Y_1)
Y.shape

(614, 10000)

In [ ]:
from tqdm import tqdm

In [ ]:
models = []

In [ ]:
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X, Y, test_size=0.2, random_state=42)

# Train multiple LightGBM models (one for each output element)
for i in tqdm(range(Y.shape[1])):

    train_data = lgb.Dataset(X_train_split, label=y_train_split[:, i])  # Train a model for each output element
    params = {
        'objective': 'regression',
        'metric': 'l2',
        'num_leaves': 31,
        'learning_rate': 0.005,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1,
        'num_threads': 16
    }
    num_round = 50
    model = lgb.train(params, train_data, num_round)
    models.append(model)

# Make predictions on the test set
y_pred = np.array([model.predict(X_test_split) for model in models]).T  # Transpose predictions

# Display the predicted values
print("Predicted values:")
print(y_pred)

 57%|█████▋    | 5671/10000 [1:04:17<49:04,  1.47it/s]


KeyboardInterrupt: ignored

In [ ]:
y_pred = np.array([model.predict(X_test_split) for model in models]).T  # Transpose predictions

# Display the predicted values
print("Predicted values:")
print(y_pred)

Predicted values:
[[ 0.51805974  0.43865342  0.57346872 ... -0.85994265  0.26441198
   1.24694183]
 [ 0.32331042  0.31008651  0.136538   ... -0.66114142  0.25374966
   1.11617382]
 [ 0.4689975   0.21278448  0.65070055 ... -0.72582075  0.30376494
   1.19806509]
 ...
 [ 0.3835998   0.22863015  0.22848228 ... -0.79639283  0.31622445
   1.15185537]
 [ 0.39135778  0.28107665  0.26182084 ... -0.74799855  0.32289163
   1.28195438]
 [ 0.60359442  0.23224388  0.23033012 ... -0.67936602  0.28009285
   2.24477927]]


In [ ]:
for i in y_pred:
  for j in i:
    print(j)

Streaming output truncated to the last 5000 lines.
0.19918411049331647
0.7704101285809812
0.3385015242070536
-0.014470686010978962
1.1196194749458952
0.3843451419850611
0.07680108926193746
0.6267499015751508
0.3495475506604192
-0.1577555711978628
0.6757106005350425
0.35080119063860543
0.06435634011623506
0.9703308724517111
1.123556876837135
0.1132801221165351
1.3213588666963578
0.546873328165625
0.6993853175984548
0.7497149676767451
0.8218266773840497
0.6986442175388673
0.30133782484555716
0.4819902639827823
0.24984874964382905
0.05751870327616973
0.06487532485765883
0.26942050661608524
0.7612953085198229
0.6554186769442604
-0.17179980135475037
0.00837829277979048
-0.02063407170454951
0.12602479911688008
0.11375688166975427
0.1576105832303287
0.23206008952665344
0.06467187776669615
-0.05709922285434061
-0.04057410652305569
0.46126999088816034
0.6578494693084098
0.19666002600945162
-0.03341544147355838
0.5294781044511594
0.33883379662219576
-0.24624760375941443
0.07869104980965666
1.091

KeyboardInterrupt: ignored

In [ ]:
y_pred.shape

(123, 5671)

In [ ]:
Y = np.vstack(Y_2)
Y.shape

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
y_pred.shape

(123, 5)

In [ ]:
y_test_split.shape

(123, 5)

In [ ]:
mse = mean_squared_error(y_pred.flatten(), y_test_split.flatten())  # Flatten arrays to 1D
print(f"Mean Squared Error: {mse}")

ValueError: ignored

In [ ]:
mse / len(y_pred)

0.015898716279861586

In [ ]:
mse = mean_squared_error(y_pred.flatten(), y_test_split.flatten())  # Flatten arrays to 1D
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 1.7687866500529135


In [ ]:
!rm -rf trained_model*

In [ ]:
!mkdir trained_models

In [ ]:
for i in range(len(models)):
  m = models[i]
  fn = "trained_models/trained_model_" + str(i) + ".txt"
  m.save_model(fn)